# Lab 2: Decision Trees & Ensembles

**Due: October 3rd at 11:59pm**

## Learning Goals
This lab is intended to introduce reading data from files, the use of discrete data, and the implementation of decision trees, both individually and in ensembles.  As with any programming assignment, you'll also be practicing and improving your general CS skills, like problem decomposition, algorithmic thinking, implementation and testing, language syntax, etc..  Here are some of the specific things you should learn while completing this assignment:

* How to load and work with data from a CSV file using the Pandas library
* How to implement the ID3 Decision Tree induction algorithm
* How to implement an ensemble of decision trees using the Bagging algorithm
* How to test algorithms using repeated hold-out validation

The assignment is presented in the form of an interactive Python notebook; it contains a mix of pre-made examples to help you understand how to do things, scaffolding with missing parts where you'll write your own code, and written short-answer questions.  Your job is to fill in answers to the written questions (which may require you to modify scaffolding code and re-run it, or may require you to write your own code and run it), as well as write the indicated functions to complete the programming portion of the assignment.  Look for the red <span style="color:red">TODO: ...</span> markers to help you spot places you'll need to complete things.

Please be sure to ***remove* the TODO markers** as you complete each aspect (i.e. don't leave a TODO that's for something you've actually done, that's poor style because it's confusing to anyone reading your code/documentation/etc.)

This lab is intended to be done with a partner (i.e. teams of two); partners will be assigned based on the survey

You'll have approximately **two weeks** to complete this lab, so be sure to start early and plan properly to get it all done in a timely fashion.  It's recommended that you start at the top and work your way down (i.e. the later parts are more difficult than the earlier ones).  As with any longer term assignment, it's a chance to train your time management skills.

# Constructing Decision Trees

Nearest Neighbor is a fine starting point, but no single algorithm is going to work well for all types of problems.  As a result, we'll need to use different algorithms for different problems.

As a case in point, Nearest Neighbor typically doesn't work very well for problems where the features are *discrete* variables (as opposed to *continuous*).  If the features are *ordinal*, they're discrete but at least ordered (e.g. "small", "medium", "large"), in which case we might be able to come up with some sort of meaningful definition of a `distance()` function.  However, for *cataogrical* features there's no ordering (e.g. "red", "blue", "green"), so really all we can say is that the feature is the same or different.

Fortunately, Decision Trees (DTs) handle discrete features very naturally; in fact, it requires a bit of extra work to get DTs to handle continuous data (it's not all *that* hard, but you won't be asked to do it for this assignment).

Note that SciKit Learn uses Numpy for all its data representations, which means it's actually not very good at handling discrete features.  As a result, while SKLearn does include a Decision Tree classifier, we can't actually make use of it as a baseline here, since we would first need to convert our data into numeric vectors to be able to have it run (and at that point, we wouldn't get the same result).  I've put the scores my reference implementation got down near the testing code so you've got a baseline anyway.

We'll also be using this as an excuse to learn about reading data from files (instead of just using data sets that come with SKLearn).

***


## Reading data

The first thing we need to do is read some data.  Previously, we just used a data set that was built in to SciKit Learn, but this time we'll be reading a data file from disk.

This week, we'll use a library called **Pandas** to read our data; it's really good for reading things like excel spreadsheets and comma-separated-value (CSV) data files.  

Like SciKit Learn, Pandas also has quite good documentation: https://pandas.pydata.org/docs/

Also, **as a reminder, the warmup lab contains a number of explanations and examples of Pandas usage**, so you may want to use code snippets from there as a starting point.

## The Data
For this part of the assignment, you'll be using a dataset covering congressional voting behavior, one looking at re-occurence of breast cancer, and one on classifying edibility of mushrooms.  The original versions are downloadable from the UCI machine learning repository: 
* https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records 
* http://archive.ics.uci.edu/ml/datasets/Mushroom
* https://archive.ics.uci.edu/ml/datasets/Breast+Cancer

For the raw data, you can just use the copy that's included with the assignment scaffolding.  However, to get the full description of the dataset (which you'll need for the written questions at the bottom), you'll need to click on those links, and then read about the  data set.  You may also need to click the "Data Set Description" link near the top of the page.  This will point you to a text file with the complete data set description (note that the filename ends in '.names', but it's really just a regular text file).

***

# This time we'll use Pandas for reading our data file (it's great for CSVs)
import pandas as pd
import numpy as np### Using Pandas to load data

We'll use the `read_csv()` method to load the files; `header=none` means that the first line of the file is a normal example (as opposed to a special header with column names), and we'll use a list of strings to specify the attribute names.  Then, we just need to give it the path to the datafile and it will be loaded as a Pandas dataframe.

Pandas dataframes work a lot like relational databases; there's a ton of things you can do with Pandas, but for today we'll keep things relatively simple.

You can read more about the `read_csv()` function here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

**HINT:** _some of the examples of Pandas usage in the warmup lab may be helpful for doing this lab!_

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np

#### Congress Data Set

In [2]:
# the data file doesn't have a headder, so we'll build an array of strings holding the column names
congressFeatureNames = ['party']
for i in range(16):
    congressFeatureNames.append('vote-'+str(i))

# this next line actually reads the file in as a Pandas dataframe
congressData = pd.read_csv('house-votes-84.data', header=None, names=congressFeatureNames)

congressData # print the data

,party,vote-0,vote-1,vote-2,vote-3,vote-4,vote-5,vote-6,vote-7,vote-8,vote-9,vote-10,vote-11,vote-12,vote-13,vote-14,vote-15
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


In [3]:
# as usual, len will give us the number of elements in a collection (in this case, how many examples are in our dataset)
len(congressData)

435

#### Mushroom Data Set

In [4]:
mushroomFeatureNames = ['edible', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 
                'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 
                'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 
                'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
    
mushroomData = pd.read_csv('agaricus-lepiota.data', header=None, names=mushroomFeatureNames)

mushroomData

23


,edible,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


#### Breast Cancer Data Set

In [5]:
breastcancerFeatureNames = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 
                            'deg-malig', 'breast', 'breast-quad', 'irradiated']
breastcancerData = pd.read_csv('breast-cancer.data', header=None, names=breastcancerFeatureNames)

breastcancerData

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiated
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


# Part 1: Questions about the data

Modify this markdown cell to add the answers to the following short-answer questions.  In doing so, make use of the code you've written above (as well as the provided code).  _Note that you won't be able to answer the final few questions until after finishing your implementation._

1. What is the _full name_ (i.e. human-readable, not filename) of the 'congress' data set?

    Congressional Voting Records

2. What sort of real-world user might be interested in a system that could successfully solve this classification problem?

    A political scientist might be interested in the voting patterns of congress members, evaluating if a democratic or republican congress member might be more or less agreeable. 

3. What are the stakes for this problem?  In other words, who might be hurt if the system makes a mistake?  How bad are the consequences?

    The stakes for this problem is relatively low as is, but if the results were to be applied on a large scale in a scheme to manipulate congress towards making a major decision, mistakes could have nationwide, even global consequences.

4. How many features (not including the class label) does each example in the data set have?

    16 features

5. How many examples does the data set contain?

    435 examples

6. What are the available class labels? Give both the encoding in the data set (i.e. the raw value) and the human-readable label associated with each value.

    republican = Republican
    
    democrat = Democrat

---

7. What is the _full name_ (i.e. human-readable, not filename) of the 'mushroom' data set?

    Mushroom
    
8. What sort of real-world user might be interested in a system that could successfully solve this classification problem?

    A biologist might be interested in the features of different types of mushrooms, possibly for research purposes. 

9. What are the stakes for this problem?  In other words, who might be hurt if the system makes a mistake?  How bad are the consequences?

    If a biologist were using this for publications or pharmaceutical researches, the stakes are higher, since it could result in research or medical accidents.
    
10. How many features (not including the class label) does each example in the data set have?

    22 features

11. How many examples does the data set contain?
    
    8124 examples

12. What are the available class labels? Give both the encoding in the data set (i.e. the raw value) and the human-readable label associated with each value.

    e = edible
    
    p = poisonous

---

13. What is the _full name_ (i.e. human-readable, not filename) of the 'breast-cancer' data set?

    Breast Cancer

14. What sort of real-world user might be interested in a system that could successfully solve this classification problem?

    An oncologist who might want to know if there is a pattern in tumor size amongst certain age demographics.

15. What are the stakes for this problem?  In other words, who might be hurt if the system makes a mistake?  How bad are the consequences?

    If the oncologist were to use this data to diagnose breast cancer, the stakes would be pretty high becuase if the system were to make a mistake, someone could undergo treatment for a cancer that don't have or not get the treatement they need. However, if the oncologist were just using the system to see if older women were more likely to have bigger or smaller tumors for a research paper they were writing, the stakes would be lower because that research would have to be fact checked and proven true in multiple iterations to be useful on a large scale.

16. How many features (not including the class label) does each example in the data set have?

    9 features

17. How many examples does the data set contain?

    286 examples

18. What are the available class labels? Give both the encoding in the data set (i.e. the raw value) and the human-readable label associated with each value.

    no-recurrence-events = no recurrent cancer events
    
    recurrence-events = recurring cancer events

---

19. For each dataset, describe in a sentence or two what the final tree looks like.  For example, how would you describe the "shape" of the tree?  Is it narrow and deep? Broad and shallow?  etc.
    
    **Congress :** The tree is narrow and deep, with many levels and a small number of branches at each node. 
    
    **Mushroom :** The tree is broad and shallow, having fewer levels but many branches departing from each node.
    
    **Breast Cancer :** The tree is broad and deep, as it has many levels and also many branches departing from each node.

20. For each dataset, what does this tell you about the problem?

    **Congress :** Given that each feature can take on either a 'yes' or 'no' value, there are only two branches at each decision point, and it needs multiple sequential decisions to classify the data.
    
    **Mushroom :** Given that most features can take on multiple values, the classification might be determined by a variety of distinct attributes near the root of the tree, without needing to delve deeply into sequential decisions.
    
    **Breast Cancer :** Given that there are both a number of features and a number of values for each feature, the classification process for this dataset might be intricate and complex. It requires navigating through numerous attributes, each potentially influenced by prior decisions, before arriving at a final prediction.
    
21. Which of the datasets do decision trees perform better on?

    Mushroom. 

22. Why?  What about that data makes it more well suited to decision tree modeling?

    Decision tree models are better suited for categorical features than for continuous ones, as they rely on distinct, categorical splits to make decisions. In the Mushroom dataset, most features are clear-cut categories, such as "odor" (with options like almond=a, anise=l, creosote=c, and so on) and "bruises" (with choices like bruises=t or no=f). On the other hand, the Breast Cancer dataset contains several features with a range of values, including continuous ones like "tumor-size" and "age", which make achieving clear splits more difficult. 

# Part 2: implementing the ID3 Decision Tree Learning algorithm

## Scaffolding

Here we'll provide some scaffolding to get you started, including some recommended function headers.  The final implementation of the algorithm is up to you; you can modify the scaffolding as needed, just try not to make the problem more complicated than it needs to be.

It's recommended (but not required) that you leave the data in the Pandas dataframe, and work with it using Pandas operators, since this will automatically make use of views (i.e. you get references rather than making deep copies of the example sets).  Whatever you use, try as always to make your code reasonably efficient (in big-O terms).

You should come up with your own development plan here; it should probably look very similar to the one for implementing the KNN algorithm, but with different functions.

Recursive## Defining Trees
Here's a basic TreeNode class; it's pretty minimal, but you don't really need anything more for this algorithm.
- `isLeaf` is meant to be a boolean, set to True if the node is a leaf node, and False otherwise
- `val` is normally the attribute (i.e. column identifier) that this node splits on, but for a leaf node it's used as the output class label instead (note that you _could_ use two separate member variables for these two things, but since any given node will only ever use one of them, it actually makes the code a bit simpler to just have the one variable)
- `children` is a dictionary, which maps from attribute values to tree nodes.  For non-leaf nodes, it should have an entry for each value that the attribute this nodes splits on can take.  Since those attribute values are used directly as the keys for the dictionary, tree traversal is very simple.  For a leaf node, it should remain empty.

In [6]:
# first we define a tree-node type
class DecisionTreeNode:
    # constructor sets up members
    def __init__(self, leaf, val):
        # initialize instance members with specified values
        self.isLeaf = leaf  # is this node a leaf node?
        if (self.isLeaf == True):
            self.label = val  # if it's a leaf, what's the class label associated with it; 
        else:
            self.splitAttribute = val   # if it's a non-leaf, what attribute do we split on
            
        self.children = {}  # list of child nodes (will be filled later if this node is not a leaf)

### Examining trees
Assuming you use the above node structure as intended, the following method should give a nice display of a decision tree; just call `printDecisionTree(root)` on the root node of the tree

In [7]:
# a recursive function to pretty-print a decision tree; 
# tries to match the visual style of decision-tree-printing in SciKit Learn
def printDecisionTree(node, indent =0, val =0):
    if (node == None): # this case should never fire for a properly built decision tree
        print('!!empty node!!')
        return
    if (node.isLeaf): # if it's a leaf node, we just need to print the class label
        for i in range(indent): # make sure we get the indent level right
            print("|   ", end="")
        print("|--- class: ", node.label)
    else: # for non-leaf nodes, we need to loop through our children and print each of them
        for val, child in node.children.items():
            for i in range(indent): # indent properly
                print("|   ", end="")
            print("|--- ", node.splitAttribute, " = ", val) # print which attribute/value this child is associated with
            printDecisionTree(child, indent+1, val) # recursively print the sub-tree for that child node

### Evaluating Trees
As usual, we don't want to use the same data for training and testing, so here's a basic function to shuffle and split our data into train and test sets, build a tree using the training data, evaluate it on the testing data, and then show the tree itself using the previous function.

**NOTE** that this function won't do anything terribly interesting until you fill in the `buildDecisionTree()` and `testDecisionTree()` functions, and you need to use the expected interface for them if you want this function to work correctly.

This is a helper function that you're encouraged to use to help test your code; example calls are given below the relevant function definition stubs.  You **shouldn't need to modify** this code unless you change the class definition given above, though it's fine to modify it if you do need to.

In [49]:
# a helper function to training & test decision tree algorithms
# params:
#   data - full dataset (will be split into test & train)
#   trainFrac - fraction of data to use for training (remainder will be used for testing)
#   seed - seed value for random number generator; defaults to 0 for repeatability, but try other values!
#   printTree - whether or not to show the tree (if False function just returns the accuracy)
def trainAndTestDT(data, trainFrac =0.7, seed =0, printTree =True):
    # let's shuffle this and split it into train and test sets:
    shuffled = data.sample(frac=1, random_state=seed) # randomly re-order the examples
        # note: the 'frac=1' means use all the examples; also note this creates a new "view", 
        # it doesn't do a deep copy, so the row index values in `shuffled` will be out of order
    trainCount = int(len(data) * trainFrac) # figure out how many examples we want in each set
    train = shuffled[:trainCount] # slice out the examples we'll use for training (again, creates a view)
    test = shuffled[trainCount:] # use the remainder for testing
    
    targetColumnName = list(train.columns)[0]
    features = list(train.columns)[1:]
    root = buildDecisionTree(train, features) # call the recursive tree building algorithm on the full data set
    acc = testDecisionTree(root, test, targetColumnName) # test the tree it gave us
    
    if (printTree):
        printDecisionTree(root) # display the tree that we got
        print("Test set accuracy: ", acc) # print accuracy

    return acc

### Repeated testing
To try to get a better sense of how reliable or consistent your algorithm is, you may want to try running it multiple times, using different splits of the data each time.  By passing in different values for the RNG seed, this function will run the above `trainAndTestDT()` function on different train/test splits and print the output each time.

This method of testing is referred to as ***repeated hold-out validation***, since we're trying to *validate* our results by *repeatedly* applying the *hold-out* testing method (i.e. randomly splitting our data into fixed-size 'train' and 'test' portions).

Again, you shouldn't need to modify this function; you'll use it later on once you've written your parts of the implementation.

In [9]:
# helper function to run the train & test function multiple times
# params:
#  data - full data set
#  repeats - number of times to repeat the experiment with a different RNG seed
def testRepeated(data, repeats):
        total=0 # accumulator variable for the accuracies so we can calculate the mean
        for j in range(repeats):
            # train and test using j as the seed for the RNG
            acc = trainAndTestDT(data, 0.7, j, False)
            print("\t Acc: %.4f" % (acc))
            total += acc
        print("Mean accuracy: %.4f" % (total / repeats)) 

***
## Recursive Tree Building from scratch
Here you will write a recursive tree-building algorithm using Information Gain to decide how to split the data at each node.  Function headers have been provided to indicate a suggested way of decomposing this problem, but it's ultimately up to you how you solve it.

_Note:_ if you use the Pandas dataframes directly, you need to be careful about iteration, because the random shuffle/split done to get training and testing data will result in sets that don't have row indexes the way you expect (the row index values are still based on the original data, not the _view_ you're using as the training set).  Thus, you may want to use something like `iterrows` (see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) so you don't get indexing errors.

However, it is usually much easier and more efficient to rely on Pandas to do the heavy lifting, rather than trying to use `iterrows` to go through the table by hand.  See the examples of using Pandas for counting and calculating ratios that were provided in the warmup lab for ideas about what might be possible.

**It is _strongly recommended_ you come up with a development plan, including a top-down design and a plan for testing your functions, _before_ you start actually writing code.**

In [10]:
# This function splits a set of examples based on a given attribute.
#       It takes in a set of examples and an identifier for which column 
#         (aka 'attribute', aka 'feature') to use for splitting.
#       returns several subsets of data, one for each value of that attribute.
#       Note that you can decide what type to return based on how the rest of your program
#       works; I used a dictionary that maps attribute values to dataframes.

import numpy as np
import pandas as pd

def split(examples, columnID):
    
    targets = examples[columnID].values
    classLabels = np.unique(targets) # get the unique labels 
    
    setDictionary = {} # put subsets keyed by unique labels in a dictionary
    for c in classLabels:
        currSet = examples[examples[columnID] == c]
        setDictionary[c] = currSet
    
    return setDictionary

In [46]:
# tests for split

uniformData = pd.read_csv('uniformTest.data', header=None, names=['label', 'feature1'])
evenSplitData = pd.read_csv('evenSplitTest.data', header=None, names=['label', 'feature1'])

evenSplitData.insert(2,"feature_2",['y','y','y','y'])
evenSplitData.insert(3,"feature_3",['y','y','y','n'])
evenSplitData.insert(4,"feature_4",['n','y','y','y'])

p = False
if (p == True):
    print(evenSplitData)
    print("-----")
    print(split(uniformData, 'feature1'))
    print("-----")
    print(split(congressData, 'vote-3'))
    print("-----")
    print(split(mushroomData, 'cap-color'))

In [48]:
from math import log

# This function calculates the class-based entropy of a set
def entropy(examples, columnID):
    
    # if subset is empty, entropy is defined as 0
    if len(examples) == 0:
        return 0
    
    setDictionary = split(examples, columnID)
    
    entropy = 0
    for key, value in setDictionary.items():
        p = abs(len(value) / len(examples)) # p : likehood of a rand item belongs to the current class
        if p != 0: # jump over the case where lg0 is not defined
            entropy += p * (log(p, 2)) # log(x, base)
            
    return -entropy

In [55]:
# tests for entropy function

#### tests on uniform data set ####
print(entropy(uniformData, 'label')) # should be 0
print(entropy(evenSplitData, 'label')) # should be 1

#### tests on empty data set ####
emptyData = pd.DataFrame({'label': []})
print(entropy(emptyData, 'label')) # should be 0 on an empty set

#### tests on real data sets ####
print(entropy(congressData, 'party'))
print(entropy(mushroomData, 'edible'))
print(entropy(breastcancerData, 'class'))

-0.0
1.0
0
0.9623080486960709
0.9990678968724604
0.8778446951746506


In [56]:
# This function calculates the information gain of a given attribute
def informationGain(examples, testColumn, targetColumn):
    
    hBefore = entropy(examples, targetColumn) # store the entropy of the pre-processed set
    
    subsetDict = split(examples, testColumn) # a dictionary of subsets keyed on split values
    
    hAfter = 0 # store the sum of entropy of subsets after the split
    for key, value in subsetDict.items():
        p = len(value) / len(examples)
        hAfter += p * entropy(value, targetColumn)
    
    infoGain = hBefore - hAfter # calculate information gain by finding the entropy difference
    return infoGain

In [64]:
# tests for the informationGain() function

#### tests on tiny data sets with predictable outcome ####
print(informationGain(uniformData, 'feature1', 'label')) # expected to be 1
print(informationGain(evenSplitData, 'feature1', 'label')) # expected to be 0

#### tests on real data set ####
print(informationGain(congressData, 'vote-5', 'party'))
print(informationGain(mushroomData, 'odor', 'edible'))
print(informationGain(breastcancerData, 'inv-nodes', 'class'))

-0.0
1.0
0.14723464628954153
0.9060749773839998
0.06899508808988597


In [65]:
# this function returns which value of the specified attribute has the most examples
from collections import Counter

def plurality(examples, columnID):
    value = examples[columnID]
    count = Counter(value[0:])
    plurality = count.most_common(1)[0][0]
    return plurality

In [66]:
# test plurality
test_attributes = list(congressData.columns)

for a in test_attributes:
    result = Counter(congressData[a])
    p = plurality(congressData, a)
    print(f"instance count : {result}")
    print(f"plurality count : {p}")

instance count : Counter({'democrat': 267, 'republican': 168})
plurality count : democrat
instance count : Counter({'n': 236, 'y': 187, '?': 12})
plurality count : n
instance count : Counter({'y': 195, 'n': 192, '?': 48})
plurality count : y
instance count : Counter({'y': 253, 'n': 171, '?': 11})
plurality count : y
instance count : Counter({'n': 247, 'y': 177, '?': 11})
plurality count : n
instance count : Counter({'y': 212, 'n': 208, '?': 15})
plurality count : y
instance count : Counter({'y': 272, 'n': 152, '?': 11})
plurality count : y
instance count : Counter({'y': 239, 'n': 182, '?': 14})
plurality count : y
instance count : Counter({'y': 242, 'n': 178, '?': 15})
plurality count : y
instance count : Counter({'y': 207, 'n': 206, '?': 22})
plurality count : y
instance count : Counter({'y': 216, 'n': 212, '?': 7})
plurality count : y
instance count : Counter({'n': 264, 'y': 150, '?': 21})
plurality count : n
instance count : Counter({'n': 233, 'y': 171, '?': 31})
plurality count : n

In [18]:
# This function create tree nodes by splitting on the "best" attribute
# takes in a set of examples to use for this node, as well as an array containing the feature names
def buildDecisionTree(currSet, currFeatures):
    
    # extract the label column from the current data set
    labelColumn = list(currSet.iloc[:, 0])
    
    targetName = list(currSet.columns)[0] # the name of the label column

    # base case (1): if currSet is a uniform set
    if (len(np.unique(labelColumn)) == 1):
        return DecisionTreeNode(True, currSet[targetName].iloc[0])
    
    # base case (2): no more features left to split on
    elif (len(currFeatures) == 0):
        return DecisionTreeNode(True, plurality(currSet, targetName))
    
    # recursive case:
    # use a maximum info gain counter to find the best feature to split on

    maxInfoGain = (-1, currFeatures[0]) # first = info gain ; second = feature name
    for f in currFeatures:
        infoGain = informationGain(currSet, f, targetName)
        if infoGain > maxInfoGain[0]:
            maxInfoGain = (infoGain, f)
    
    # split currSet on the best feature
    subsets = split(currSet, maxInfoGain[1])
    
    # update remainingFeatures by removing the feature that we just went through
    newCurrFeatures = [f for f in currFeatures if f != maxInfoGain[1]]
    
    # build a tree node from the current set
    node = DecisionTreeNode(False, maxInfoGain[1])
    
    # build the children for this node with recursion
    for f, subset in subsets.items():
        node.children[f] = buildDecisionTree(subset, newCurrFeatures)
        
    return node

In [68]:
import random  # if we have no information, we'll select randomly

# This function classifies an example using a tree with recursion
def classify(node, example):
    # case 1 : if the node is empty
    if node == None:
        print("the node is empty =^OwO^=")
        return 
    
    # case 2 : if the node is a uniform set and we can make a decision
    if node.isLeaf:
        return node.label
    
    # case 3 : the current node is intermediate, and we will traverse to the child that is keyed 
    # on a value that our example has
    else:
        splitOn = node.splitAttribute # get the feature on which this set is split on
        if splitOn not in example: # error checking! since splitOn might not be in example
            thisVal = random.choice(list(node.children.keys())) # if not, select a random value to be the key
        else:
            thisVal = example[splitOn]
            
        if thisVal not in node.children: # if the potential key does not exist, choose a random one from the existing keys
            thisVal = random.choice(list(node.children.keys()))
            
        return classify(node.children[thisVal], example)

In [69]:
# tests for classify()

#### tests on tiny data sets ####
remainingFeatures = [f for f in list(evenSplitData.columns) if f != "label"]
root = buildDecisionTree(evenSplitData, remainingFeatures)
test = {
    "feature1" : "n",
    "feature_2" : "y",
    "feature_3" : "y",
    "feature_4" : "y"
}
print(classify(root, test))
print(evenSplitData)
printDecisionTree(root)

#### tests on real data sets are left in the next section ####

blue
  label feature1 feature_2 feature_3 feature_4
0   red        y         y         y         n
1   red        y         y         y         y
2  blue        n         y         y         y
3  blue        n         y         n         y
|---  feature1  =  n
|   |--- class:  blue
|---  feature1  =  y
|   |--- class:  red


In [70]:
# This function iterates through examples in a test set, applies 
# the decision tree to each one, and then returns the overall mean accuracy of 
# the tree on that test set.  Remember that if testSet is a pandas dataframe, 
# you can't iterate using a "normal" 'for i in range(len())' loop; try using 'testSet.iterrows()'
from sklearn.model_selection import train_test_split

def testDecisionTree(root, testSet, label):
    # create a correct answer counter that tracks the number of correct predictions
    correct = 0
    
    for index, row in testSet.iterrows():
        trueLabel = row[label]
        prediction = classify(root, row)
        if prediction == trueLabel:
            correct += 1
    
    acc = correct / len(testSet)
    print(f"the number we got correct is *{correct}*, and the accuracy is *{acc}*")
    return acc

***
#### Tests on Mushroom data:
- with default parameters, should be 100% accuracy
- should likely be 100% for each repeat
***

In [72]:
print("DT on Mushroom data:")
trainAndTestDT(mushroomData) 

DT on Mushroom data:
the number we got correct is *2438*, and the accuracy is *1.0*
|---  odor  =  a
|   |--- class:  e
|---  odor  =  c
|   |--- class:  p
|---  odor  =  f
|   |--- class:  p
|---  odor  =  l
|   |--- class:  e
|---  odor  =  m
|   |--- class:  p
|---  odor  =  n
|   |---  spore-print-color  =  b
|   |   |--- class:  e
|   |---  spore-print-color  =  h
|   |   |--- class:  e
|   |---  spore-print-color  =  k
|   |   |--- class:  e
|   |---  spore-print-color  =  n
|   |   |--- class:  e
|   |---  spore-print-color  =  o
|   |   |--- class:  e
|   |---  spore-print-color  =  r
|   |   |--- class:  p
|   |---  spore-print-color  =  w
|   |   |---  habitat  =  d
|   |   |   |---  gill-size  =  b
|   |   |   |   |--- class:  e
|   |   |   |---  gill-size  =  n
|   |   |   |   |--- class:  p
|   |   |---  habitat  =  g
|   |   |   |--- class:  e
|   |   |---  habitat  =  l
|   |   |   |---  cap-color  =  c
|   |   |   |   |--- class:  e
|   |   |   |---  cap-color  =  n
|  

1.0

In [73]:
testRepeated(mushroomData, 10) 

the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
the number we got correct is *2438*, and the accuracy is *1.0*
	 Acc: 1.0000
Mean accuracy: 1.0000


***
#### Tests on Congress data:
 - with default parameters, should be 91.6% accuracy
 - mean of 10 repeats should be something like 92.7%
***

In [74]:
print("DT on Congress data:")
trainAndTestDT(congressData)

DT on Congress data:
the number we got correct is *120*, and the accuracy is *0.916030534351145*
|---  vote-3  =  ?
|   |---  vote-11  =  ?
|   |   |--- class:  republican
|   |---  vote-11  =  n
|   |   |--- class:  democrat
|   |---  vote-11  =  y
|   |   |--- class:  republican
|---  vote-3  =  n
|   |--- class:  democrat
|---  vote-3  =  y
|   |---  vote-10  =  ?
|   |   |--- class:  republican
|   |---  vote-10  =  n
|   |   |---  vote-14  =  ?
|   |   |   |--- class:  republican
|   |   |---  vote-14  =  n
|   |   |   |--- class:  republican
|   |   |---  vote-14  =  y
|   |   |   |---  vote-9  =  n
|   |   |   |   |---  vote-15  =  ?
|   |   |   |   |   |---  vote-1  =  n
|   |   |   |   |   |   |--- class:  democrat
|   |   |   |   |   |---  vote-1  =  y
|   |   |   |   |   |   |--- class:  republican
|   |   |   |   |---  vote-15  =  n
|   |   |   |   |   |--- class:  republican
|   |   |   |   |---  vote-15  =  y
|   |   |   |   |   |--- class:  democrat
|   |   |   |---  vot

0.916030534351145

In [75]:
testRepeated(congressData, 10)

the number we got correct is *120*, and the accuracy is *0.916030534351145*
	 Acc: 0.9160
the number we got correct is *119*, and the accuracy is *0.9083969465648855*
	 Acc: 0.9084
the number we got correct is *124*, and the accuracy is *0.9465648854961832*
	 Acc: 0.9466
the number we got correct is *119*, and the accuracy is *0.9083969465648855*
	 Acc: 0.9084
the number we got correct is *120*, and the accuracy is *0.916030534351145*
	 Acc: 0.9160
the number we got correct is *125*, and the accuracy is *0.9541984732824428*
	 Acc: 0.9542
the number we got correct is *125*, and the accuracy is *0.9541984732824428*
	 Acc: 0.9542
the number we got correct is *119*, and the accuracy is *0.9083969465648855*
	 Acc: 0.9084
the number we got correct is *121*, and the accuracy is *0.9236641221374046*
	 Acc: 0.9237
the number we got correct is *122*, and the accuracy is *0.9312977099236641*
	 Acc: 0.9313
Mean accuracy: 0.9267


***
#### Tests on Breast Cancer data:
- mean of 10 repeats should be something like ~65%
***

In [76]:
print("DT on Breast Cancer Data Set")
trainAndTestDT(breastcancerData)

DT on Breast Cancer Data Set
the number we got correct is *53*, and the accuracy is *0.6162790697674418*
|---  inv-nodes  =  0-2
|   |---  tumor-size  =  0-4
|   |   |---  age  =  30-39
|   |   |   |---  menopause  =  premeno
|   |   |   |   |---  node-caps  =  no
|   |   |   |   |   |---  deg-malig  =  2
|   |   |   |   |   |   |---  breast  =  right
|   |   |   |   |   |   |   |---  breast-quad  =  central
|   |   |   |   |   |   |   |   |---  irradiated  =  no
|   |   |   |   |   |   |   |   |   |--- class:  no-recurrence-events
|   |   |---  age  =  40-49
|   |   |   |--- class:  no-recurrence-events
|   |   |---  age  =  50-59
|   |   |   |--- class:  no-recurrence-events
|   |---  tumor-size  =  10-14
|   |   |--- class:  no-recurrence-events
|   |---  tumor-size  =  15-19
|   |   |---  menopause  =  ge40
|   |   |   |--- class:  no-recurrence-events
|   |   |---  menopause  =  lt40
|   |   |   |--- class:  no-recurrence-events
|   |   |---  menopause  =  premeno
|   |   |   |---

0.6162790697674418

In [77]:
testRepeated(breastcancerData, 10) 

the number we got correct is *51*, and the accuracy is *0.5930232558139535*
	 Acc: 0.5930
the number we got correct is *60*, and the accuracy is *0.6976744186046512*
	 Acc: 0.6977
the number we got correct is *57*, and the accuracy is *0.6627906976744186*
	 Acc: 0.6628
the number we got correct is *53*, and the accuracy is *0.6162790697674418*
	 Acc: 0.6163
the number we got correct is *61*, and the accuracy is *0.7093023255813954*
	 Acc: 0.7093
the number we got correct is *59*, and the accuracy is *0.686046511627907*
	 Acc: 0.6860
the number we got correct is *57*, and the accuracy is *0.6627906976744186*
	 Acc: 0.6628
the number we got correct is *59*, and the accuracy is *0.686046511627907*
	 Acc: 0.6860
the number we got correct is *58*, and the accuracy is *0.6744186046511628*
	 Acc: 0.6744
the number we got correct is *45*, and the accuracy is *0.5232558139534884*
	 Acc: 0.5233
Mean accuracy: 0.6512


***
# Part 3: Ensembles of Trees

Decision trees are nice in many ways, but they have a tendency to overfit their training data.  One way to combat this tendency is to train multiple trees and then combine the results.

Here, you'll write code to create an ensemble of decision trees using **bagging**, short for **b**ootstrap **agg**regat**ing**.  This method works by creating several different training sets, training a decision tree on each set, and then letting the resulting trees "vote" on the correct answer to a novel query.

A function to train an ensemble should take a number of trees $k$ and the fraction of the training data $f$ to use for each tree.  It should then call your `buildDecisionTree()` function $k$ times, each time using $f$ percent of the full training set selected at random _with replacement_.  If you're using Pandas, you can look at the `sample()` method (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html), which will do most of the work for you.

Note that you should not have to actually modify your `buildDecisionTree()` function; instead, you are just calling it repeatedly with different data sets, and storing those trees to make a 'forest'.

To classify a novel example using an ensemble, you'll need to call the single-tree classify function on that example for each tree in the ensemble; then, pick the label that got the most votes (i.e. the plurality choice) as the output for the ensemble as a whole.

Finally, you'll want to write a function to test your ensemble on the testing set and then try it out (you may want to copy and modify the `trainAndTestDT()` function given above).

Again, it is _**strongly recommended**_ you come up with a development plan, including a top-down design and a plan for testing your functions, _before_ you start actually writing code.

Note that you should typically see an improvement of a couple percentage points on average for the Congress and Breast Cancer data when using bagging (e.g. for Congress I got an average of 94.5% over 10 runs using 5 trees per ensemble and 0.5 as my bagging fraction, as opposed to the 92.7% for single DTs).  The mushroom data should already be getting 100% accuracy with a single tree, so there's really no room for improvement on that one.

In [108]:
def getVotes(forest, example):
    predictions = {} # record the number of predictions that our model got correct
    for tree in forest:
        p = classify(tree, example)
        if p not in predictions:
            predictions[p] = 1
        else:
            predictions[p] += 1
            
    return max(predictions, key=predictions.get) # return the most common vote

def trainAndTestEnsemble(data, k = 5, f = 0.5, trainFrac = 0.7, seed = 0, printTree = False):
    # process our original data set by randomly shuffling
    shuffled = data.sample(frac=1, random_state=seed)
    trainCount = int(len(data) * trainFrac)
    
    # re-organize the original set into 2 sets, the training set and the testing set
    train = shuffled[:trainCount]
    test = shuffled[trainCount:]
    
    # get a forest of trained trees
    forest = []
    for i in range(k): # randomly pick k subsets from the training set and train each tree individually
        subset = train.sample(frac=f, replace=True)
        forest.append(buildDecisionTree(subset, list(subset.columns)[1:]))
        
    labelColName = list(test.columns)[0]
    correct = 0
    for index, row in test.iterrows():
        trueLabel = row[labelColName]
        nextRow = row[row.index != labelColName]
        prediction = getVotes(forest, nextRow)
        if trueLabel == prediction:
            correct += 1
            
    accuracy = correct / len(test)
            
    return accuracy

In [109]:
print("Ensemble on Congress Data")
testRepeatedEnsemble(congressData, 10)
print("----------")
print("Ensemble on Mushroom Data")
testRepeatedEnsemble(mushroomData, 10)
print("----------")
print("Ensemble on Breast Cancer Data")
testRepeatedEnsemble(breastcancerData, 10)
print("----------")

Ensemble on Congress Data
	 Acc: 0.9466
	 Acc: 0.9084
	 Acc: 0.9618
	 Acc: 0.9160
	 Acc: 0.9542
	 Acc: 0.9618
	 Acc: 0.9237
	 Acc: 0.9313
	 Acc: 0.9389
	 Acc: 0.9466
Mean accuracy: 0.9389
----------
Ensemble on Mushroom Data
	 Acc: 0.9988
	 Acc: 1.0000
	 Acc: 1.0000
	 Acc: 1.0000
	 Acc: 0.9996
	 Acc: 1.0000
	 Acc: 1.0000
	 Acc: 0.9996
	 Acc: 1.0000
	 Acc: 0.9992
Mean accuracy: 0.9997
----------
Ensemble on Breast Cancer Data
	 Acc: 0.6860
	 Acc: 0.6628
	 Acc: 0.6860
	 Acc: 0.6860
	 Acc: 0.7326
	 Acc: 0.7326
	 Acc: 0.6279
	 Acc: 0.6512
	 Acc: 0.6395
	 Acc: 0.5930
Mean accuracy: 0.6698
----------


In [110]:
# helper function to run the train & test ensemble bagging multiple times
# params:
#  data - full data set
#  repeats - number of times to repeat the experiment with a different RNG seed
def testRepeatedEnsemble(data, repeats):
        total=0 # accumulator variable for the accuracies so we can calculate the mean
        for j in range(repeats):
            # train and test using j as the seed for the RNG
            acc = trainAndTestEnsemble(data, 5, 0.5, 0.7, j, False)
            print("\t Acc: %.4f" % (acc))
            total += acc
        print("Mean accuracy: %.4f" % (total / repeats)) 

***
## Optional extension

If you want to take things one step further, try implementing the full Random Forest algorithm, which lets you also choose a fraction of the _features_ to consider when creating a tree.  You can either do this by selecting features on a per-tree basis or a per-node basis; for my reference implementation, it was much easier to do it on a per-node basis (in fact, it only required a couple of extra lines of code to be added to my `buildDecisionTree()` function).

Note that for the Congress data set this likely won't boost your performancy by that much (you're unlikely to get more than another 1% on average), but for some problems it makes a more significant difference.
***

In [30]:
# OPTIONALLY: put extra code below if you want to do the extension